# Introduction

The aim of our project is to find out whether the product created from two designated molecules is stable. Indeed, our vision is to ask a user to draw two molecules of his choice. Thanks to the "reaction_utils" template, a product is created. To find out whether it was stable and whether the reaction was spontaneous, we also chose to calculate free enthalpies.

Free enthalpy, or Gibbs free energy, ΔG is a thermodynamic function that determines whether a chemical reaction can occur spontaneously at constant temperature and pressure. It is calculated by the formula ΔG=ΔH-TΔS where H is enthalpy [J], T is the temperature [K] and S is the entropy [J/K]. A reaction is spontaneous if the variation in free enthalpy is negative. In our case,  If the enthalpy of the sum of the reactants is greater than that of the product, then the latter is stable. This concept is therefore essential for predicting chemical equilibrium and optimizing the conditions of chemical reactions, whether industrial or biological. Free enthalpy thus enables us to better understand the direction and feasibility of chemical transformations.


# Project, Problems and Solutions

## 1. Path to find the energy

In this project, several aspects were analyzed. First, the goal was to analyze the free energy of each molecule. Indeed, if the sum of the reactants enthalpy is greater than the product, the reaction is spontaneous. On the contrary, the product is not stable and the reaction will not occur.
Therefore, the enthalpy was calculated using the following steps : when the user draw its reactants, we created a code which converted the draw reactants into smiles. Then, a function was programmed to convert smiles into into an XYZ file. From this latter, rdkit generates 3D coordinates. Finally, an other function using XTB and morfeus converted the latter in energy. 

Another part also involved creating the product using the template "reaction_utils" and follow the same steps as mentioned to get its free energy. 
All this stages can be summaryze with this chart : 

Molecule ( Smiles “CCOC…”) → Convert into 3D (xyz) → Found the energy using XTB and Morfeus

problems

solutions

## 2. Streamlit

Another point of this project was to create an interface. Inspired by lecture 12, streamlit and ketcher were programmed. So, when the user draws the two reagents, they can see their drawing as well as the smiles, the 3D visualization and the energy of his molecules. The product is also illustrated with the same features.

One of the main challenge of this part was to obtain the product into the screen interface. Indeed, for aesthetic reasons, we tried to get the product on the screen created with ketcher. However, when the product is created once, it is not possible to change the reagents again and see the new molecule. In fact, the page has to be reinitialized. 
Secondly, we've chosen to create a site that's easy to navigate for the reader. For example, the user must click on the “Draw molecule 2” button after drawing the first molecule. This button was created because if the two drawing interfaces were in place from the start, the page would “jump” directly to the Molecule 2 section. Unwrapping the site in several stages with buttons enabled us to solve this problem.




# 3. Results and Discussion




